Modified TorchVision 0.3 Object Detection finetuning tutorial (https://colab.research.google.com/github/pytorch/vision/blob/temp-tutorial/tutorials/torchvision_finetuning_instance_segmentation.ipynb), then my torchvision_finetuning_instance_segmentation.ipynb,
then fixed usage of segmentation


In [ ]:
%%shell

cd /content
git clone https://github.com/pytorch/vision.git
pip install cython
# Install pycocotools, the version by default in Colab
# has a bug fixed in https://github.com/cocodataset/cocoapi/pull/354
pip install -U 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'

In [ ]:
!nvidia-smi

from google.colab import drive

drive.mount('/home/gdrive')

%cd "/home/gdrive/My Drive/INIRS"
!cp -r Images /content/
    

In [ ]:
import os

# download the Penn-Fudan dataset
if 0: # not os.path.exists('PennFudanPed'):
    !wget https://www.cis.upenn.edu/~jshi/ped_html/PennFudanPed.zip .
    # extract it in the current folder
    !unzip PennFudanPed.zip
    !rm PennFudanPed/PNGImages/*[1-9].*
    !rm PennFudanPed/PedMasks/*[1-9]_mask.*
    # !rm PennFudanPed/PNGImages/*[4-9]0.*
    # !rm PennFudanPed/PedMasks/*[4-9]0_mask.*
    # !ls PennFudanPed/PNGImages

In [ ]:
%%shell

# # Download TorchVision repo to use some files from
# # references/detection
# git clone https://github.com/pytorch/vision.git
pwd
# cd vision
# git checkout v0.3.0

# cp references/segmentation/utils.py ../
# cp references/segmentation/transforms.py ../
# # cp references/segmentation/coco_eval.py ../
# # cp references/segmentation/train.py ../     # Changed and loaded manually
# cp references/segmentation/coco_utils.py ../

In [ ]:
import os
import numpy as np
try:
    from scipy.misc import imsave
except:
    from imageio import imsave 
import torch
import torch.utils.data
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib

# sys.path.append('PyTorch')
from PyTorch.PyTChipNets import *
from ChipJupyterNotUtils import *

%matplotlib inline 
%load_ext autoreload
%autoreload 2

outDir = 'NetLogs'
weightsFileNameTempl = outDir + '/ChipWeights_Epoch%d.h5'

In [ ]:
def printProgress(str):
    with open(outDir + '/progress.log', 'a') as file:
        file.write(str + '\n')

In [ ]:
# dataset = PennDataset('PennFudanPed/')
# dataset = ChipDataset('/content')
# dataset[0]
# len(dataset)
# for item in dataset:
#     print(item[0].size)

    # print(get_transform(False)(item[0]))
#     print(item[1]['masks'].shape)
# dataset[5][1]['masks'].shape

So we can see that by default, the dataset returns a `PIL.Image` and a dictionary
containing several fields, including `boxes`, `labels` and `masks`.

In [ ]:
# !cp /usr/local/lib/python3.6/dist-packages/torchvision/models/detection/mask_rcnn.py .
# !pwd
# !find / -name '*mask_rcnn*'
# !pip freeze|grep torch
%ls -l PyTorch/*Chip*

In [ ]:
from PyTorch import train
from PyTorch.train import train_one_epoch, criterion, evaluate
from PyTorch import utils

### Putting everything together

In [ ]:
class DeepOptions:
    if 0:
        basePlaneCount=64
        batchSize=4
    else:
        basePlaneCount=16
        batchSize=1
        
# dataset = PennFudanDataset('PennFudanPed', get_transform(train=True))
# dataset_test = PennFudanDataset('PennFudanPed', get_transform(train=False))
dataset = ChipDataset('/content/Images', get_transforms(train=True), DeepOptions.batchSize)
dataset_test = ChipDataset('/content/Images', get_transforms(train=False))

# split the dataset in train and test set
torch.manual_seed(1)
indices = torch.randperm(len(dataset)).tolist()
if len(indices) > 50:
    testImageCount = 50
else:
    testImageCount = len(indices) // 3

if testImageCount > 0:
    dataset = torch.utils.data.Subset(dataset, indices[:-testImageCount])
    dataset_test = torch.utils.data.Subset(dataset_test, indices[-testImageCount:])
else:
    dataset = torch.utils.data.Subset(dataset, indices)
    dataset_test = torch.utils.data.Subset(dataset_test, indices)
train_sampler = torch.utils.data.RandomSampler(dataset)
test_sampler = torch.utils.data.SequentialSampler(dataset_test)

# define training and validation data loaders
data_loader = torch.utils.data.DataLoader(
    dataset, batch_size=16, num_workers=4,  # shuffle=True, 
    sampler=train_sampler, collate_fn=utils.collate_fn)

data_loader_test = torch.utils.data.DataLoader(
    dataset_test, batch_size=1, num_workers=4,  # shuffle=False, 
    sampler=test_sampler, collate_fn=utils.collate_fn)

In [ ]:
# import segmentation_models_pytorch 

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# device = torch.device('cpu')

# person or not # our dataset has two classes only - background and person
num_classes = 1

# get the model using our helper function
# model = get_instance_segmentation_model(num_classes)
model = ChipNet(num_classes, basePlaneCount=DeepOptions.basePlaneCount)
# model = segmentation_models_pytorch .Unet("resnet18", encoder_weights="imagenet", classes=1, activation=None)
# print(model)

# move model to the right device
model.to(device)
epochNum = 0

In [ ]:
!mkdir $outDir
!mkdir $outDir/src
!cp *.py $outDir/src
!cp *.ipynb $outDir/src
!cp -r PyTorch $outDir/src
%pwd

In [ ]:
# pick one image from the test set
# img, _ = dataset_test[0]
img, target = dataset[0]
# print(target['mask'].max())
# plt.imshow(np.array(input.numpy().transpose(1, 2, 0), dtype=np.float32))
plt.imshow(np.array(np.squeeze(target.numpy(), 0), dtype=np.float32))
# img.to(device)
img.shape, np.mean(img.numpy(), axis=(1, 2)), np.std(img.numpy(), axis=(1, 2))
# target.shape, np.mean(target.numpy(), axis=(1, 2)), np.std(target.numpy(), axis=(1, 2))

In [ ]:
def printWeightsStats(model):
    for layerName in model.getAllLayers():
        weights = model.getMultWeights(layerName)
        infoStr = '%s: %s, %.5g, %.5g, %.5g, %.5g' % (layerName, str(weights.shape), 
                    weights.min(), weights.max(), np.mean(weights), np.std(weights))
        print(infoStr)
        printProgress(infoStr)

In [ ]:
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005,   # 0.005
                            momentum=0.9, weight_decay=0.0005)

# and a learning rate scheduler which decreases the learning rate by 10x every 3 epochs
# lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=20,  # 3
#                                                gamma=0.5)
lr_scheduler = torch.optim.lr_scheduler.CyclicLR(optimizer, step_size_up=20,
                                                 base_lr=0.001, max_lr=0.01)

In [ ]:
# let's train it for 10 epochs
num_epochs = 500

i = 1
t = 1
for _ in range(num_epochs):
    # train for one epoch, printing every 10 iterations
    model.eval()
    with torch.no_grad():
        if len(img.shape) == 3:
            img.unsqueeze_(0)
        prediction = model(img.to(device))
        prediction = prediction[0].cpu().numpy().transpose(1, 2, 0)
        # prediction[prediction > 1] = 1
        # prediction[prediction < 0] = 0
        print(prediction.shape, prediction.dtype, prediction.min(), np.mean(prediction), prediction.max())
    fig = plt.imshow(np.squeeze(prediction, 2),
               vmin=-1, vmax=2, cmap='rainbow');
    plt.colorbar()
    plt.show()

 # def train_one_epoch(model, criterion, optimizer, data_loader, lr_scheduler,
 # device, epochNum, print_freq):
    
    metric_logger = train_one_epoch(model, criterion, optimizer, data_loader, lr_scheduler, \
                    device, epochNum, print_freq=10)
    if epochNum > 0 and epochNum % 10 == 0:
        print(weightsFileNameTempl % epochNum)
        model.saveState(weightsFileNameTempl % epochNum)
        prediction[prediction < 0] = 0
        prediction[prediction > 2] = 2
        imsave(outDir + '/Pred_Epoch%d.png' % epochNum, prediction, format='png')
        # printProgress('Epoch %d: learn. rate %.3g, loss %.7g' % \
        #                 (epochNum, optimizer.param_groups[0]["lr"])) 
        showWeights(model.getMultWeights('conv1'), 8)
        plt.show()
    printProgress('Epoch %d: %s' % \
                    (epochNum, str(metric_logger)))
    printWeightsStats(model)
    # update the learning rate
    # lr_scheduler.step()
    # evaluate on the test dataset
    # evaluate(model, data_loader_test, device, num_classes)
    # print(x)
    epochNum += 1

In [ ]:
 fig = plt.imshow(Image.fromarray(np.squeeze(prediction, 2)),
               vmin=-0.01, vmax=0.01, cmap='Greys_r');

In [ ]:
print(train.i.shape, train.t.shape, train.i[0, :, :, 0])
train.t.squeeze(1).shape
nn.BCELoss(train.i, train.t)


In [ ]:
input = torch.randn(3, 50, requires_grad=True)
target = torch.randint(50, (3,), dtype=torch.int64)
loss = nn.functional.cross_entropy(input, target)
# input.shape
# target
loss

In [ ]:
nn.functional.cross_entropy(train.i, train.t)

In [ ]:
# put the model in evaluation mode
model.eval()
with torch.no_grad():
    prediction = model([img.to(device)])

Printing the prediction shows that we have a list of dictionaries. Each element of the list corresponds to a different image. As we have a single image, there is a single dictionary in the list.
The dictionary contains the predictions for the image we passed. In this case, we can see that it contains `boxes`, `labels`, `masks` and `scores` as fields.

In [ ]:
Image.fromarray(prediction[0]['masks'][0, 0].mul(255).byte().cpu().numpy())

In [ ]:
Image.fromarray(img.mul(255).permute(1, 2, 0).byte().numpy())

And let's now visualize the top predicted segmentation mask. The masks are predicted as `[N, 1, H, W]`, where `N` is the number of predictions, and are probability maps between 0-1.